In [0]:
data="/FileStore/tables/IBM.csv"
df=spark.read.format("csv").option("header","true").option("inferSchema","true").load(data)
#df.show()
from pyspark.sql.window import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
win=Window.orderBy(col("Date").desc())
def grade(num):
    num=float(num or 0)
    if num>0.01 and num<0.1:
        return "little bit profit"
    elif num>0.9:
        return "high profit"
    elif num>=0.1 and num<=0.9:
        return "ok profit"
    else:
        return "loss"
#convert function into spark understandable format.
gudf=udf(grade)
ndf=df.withColumn("lead",lead(col("Close")).over(win)).withColumn("diff",col("Close")-col("lead")).withColumn("status",when(col("diff")>col("high"),col("low"))).withColumn("progress",gudf(col("diff")))
ndf.show()

+----------+----------+----------+----------+----------+----------+-------+----------+--------------------+------+-----------+
|      Date|      Open|      High|       Low|     Close| Adj Close| Volume|      lead|                diff|status|   progress|
+----------+----------+----------+----------+----------+----------+-------+----------+--------------------+------+-----------+
|2024-02-29|186.149994|186.850006|184.690002|185.029999|185.029999|6457100|185.300003|-0.27000400000000013|  null|       loss|
|2024-02-28|184.630005|185.369995|183.550003|185.300003|185.300003|3216300|184.869995| 0.43000800000001504|  null|  ok profit|
|2024-02-27|184.160004|185.130005|182.619995|184.869995|184.869995|3641400|184.130005|  0.7399899999999775|  null|  ok profit|
|2024-02-26|185.600006|186.130005|184.059998|184.130005|184.130005|4620800|185.720001|  -1.589995999999985|  null|       loss|
|2024-02-23|184.899994|186.460007|184.570007|185.720001|185.720001|3432900|184.210007|   1.509994000000006|  nu